# Scheduler K8S Test

Assumes you have a Kind cluster running with metallb. No other dependencies required.
You can use [Seldon Ansible Kind Playbook](https://github.com/SeldonIO/ansible-k8s-collection/blob/master/playbooks/kind.yaml)

## Setup

* `make kind-image-install-all`
* `make deploy` 


In [1]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [2]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

## No Auth Example

In [3]:
!grpcurl -d '{"model":{"name":"iris","uri":"gs://seldon-models/mlserver/iris",\
                      "requirements":["sklearn"],\
                      "memoryBytes":50000,\
                      "replicas":2}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [4]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "serverName": "mlserver",
  "modelReplicaState": {
    "0": "Loaded",
    "1": "Loaded"
  }
}


In [5]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55690ff4a4f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55690ff4a4f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Tue, 30 Nov 2021 17:27:20 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1101
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"b3792bbf-1291-49d1-aae2-769db6b766d4","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [6]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [7]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "100000",
      "availableMemoryBytes": "50000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "50000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    }
  ]
}


In [8]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


In [9]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x5579448c94f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x5579448c94f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)







* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


In [10]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

ERROR:
  Code: FailedPrecondition
  Message: Failed to find model iris


In [11]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    }
  ]
}


## Inline RClone Config Example

Before running this 
  * install minio in a kind cluster with Ansible.
  * Update ip address for exposed minio ip address in example below
  
Create rclone.conf changing the ip address for minio as appropriate

```
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://172.18.255.1:9000
```
  
 Copy iris model to minio
 
 ```
 rclone --config ./rclone.conf copy mlrepo/iris s3://test
 ```

In [12]:
!grpcurl --format text -d '\
    model { \
     name: "iris" \
     version: "1" \
     uri: "s3://models/iris" \
     storageConfig: { \
           storageRcloneConfig: "{\"type\":\"s3\",\"name\":\"s3\",\"parameters\":{\"provider\":\"minio\",\"env_auth\":\"false\",\"access_key_id\":\"minioadmin\",\"secret_access_key\":\"minioadmin\",\"endpoint\":\"http://172.18.255.1:9000\"}}" \
     } \
     requirements: "sklearn" \
     memoryBytes: 500 \
     replicas: 1}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [13]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "100000",
      "availableMemoryBytes": "99500"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    }
  ]
}


In [14]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55e0811654f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55e0811654f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Tue, 30 Nov 2021 17:34:56 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1024
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"e62391b9-6e38-414d-8a4f-01dde5d399e3","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [15]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [16]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


## K8S Secret RClone Config Example

 * Update endpoint for minio below

In [17]:
%%writefile minio-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  namespace: seldon-mesh
type: Opaque
stringData:
  s3: |
    type: s3
    name: s3
    parameters:
      provider: minio
      env_auth: false
      access_key_id: minioadmin
      secret_access_key: minioadmin
      endpoint: http://172.18.255.1:9000

Overwriting minio-secret.yaml


In [18]:
!kubectl apply -f minio-secret.yaml

secret/minio-secret created


In [19]:
!grpcurl --format text -d '\
    model { \
     name: "iris" \
     version: "1" \
     uri: "s3://models/iris" \
     storageConfig: { \
      storageSecretName: "minio-secret" \
      } \
     requirements: "sklearn" \
     memoryBytes: 500 \
     replicas: 1}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [20]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "memory": "100000",
      "availableMemoryBytes": "99500"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    },
    {
      "memory": "100000",
      "availableMemoryBytes": "100000"
    }
  ]
}


In [21]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "Host: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55e2436544f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55e2436544f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: iris
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Tue, 30 Nov 2021 17:37:48 GMT
< server: envoy
< content-length: 199
< content-type: application/json
< x-envoy-upstream-service-time: 1234
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"v0.1.0","id":"45ba843e-8447-4713-8389-8a9c0be52739","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [22]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header Seldon:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "v0.1.0",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [23]:
!grpcurl -d '{"name":"iris"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}
